# Python

-
*
+
@list-3

[TOC]


## Anaconda

[下载地址](https://www.anaconda.com/download/#linux)

[安装步骤](https://docs.anaconda.com/anaconda/install/linux)

* 安装
    - bash ~/Downloads/Anaconda3-5.1.0-Linux-x86_64.sh
    - echo 'export PATH="~/anaconda2/bin:$PATH"' >> ~/.bashrc
    - source ~/.bashrc
    
* IDE使用anaconda
    - pycharm
        setting--Project Interpreter--add local--Conda Environment--ex

### Conda

[官方文档](https://conda.io/docs/user-guide/tasks/index.html)

[查询在线包链接](https://anaconda.org/)

* 查看内容
    - conda list
    - conda list -n env_name
    
    - conda -V
    - conda -h //查看帮助
    - conda info -e //查看所以安装的环境
    - python -V
* 更新所有包
    - conda upgrade -all
* install
    - conda install package_name
    
    - conda install numpy=1.13.3
    - conda install --name [环境名称] beautifulsoup4
* remove
    - conda remove package_names
    - conda env remove -n xxx //删除指定的环境
    - conda remove -n xxx --all
* update
    - conda update --prefix /home/xxxx/anaconda3 anaconda
    - conda update -n base conda
    - conda update package_name
    
* search
    - conda search search_term //可模糊查询
* create
    - conda create -n env_name package_names //env_name 设置环境
    + conda create --name python36 python=3.6
    + conda create -n py3 python=3 //可能是最新的包
    
    + conda info -e //确定创建好
    - conda create -n py3 pandas //创建环境py3的环境并其中安装numpy
    
    - conda create --name py3 --clone base //从其他环境拷贝到新的环境
* activate
    - source activate py3 //进入环境
* deactivate
    - source deactivate
* 镜像
    - ~/.condarc (Linux/Mac)
        ```
        channels
        https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free/
        https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main/
        defaults
        show_channel_urls: yes
        ```       
    - ~/.pip/pip.conf (Linux/Mac)
        ```
        global
        trusted-host =  pypi.douban.com
        index-url = http://pypi.douban.com/simple
        ```
  
    

### jupyter notebook

* 快捷键
    - Esc + F 在代码中查找、替换
    - Esc + O 在cell和输出结果间切换
    - Shift + J 或 Shift + Down 选择下一个cell
    - Shift + K 或 Shift + Up 选择上一个cell
    - Shift + M 合并cell

## pip

- AttributeError: module 'pip' has no attribute 'main'
    * 安装目录helpers/packaging_tool.py
    ```
def do_install(pkgs):
    try:
        # import pip
        try:
            from pip._internal import main
        except Exception:
            from pip import main
    except ImportError:
        error_no_pip()
    return main(['install'] + pkgs)


def do_uninstall(pkgs):
    try:
        # import pip
        try:
            from pip._internal import main
        except Exception:
            from pip import main
    except ImportError:
        error_no_pip()
    return main(['uninstall', '-y'] + pkgs)
    ```


## 命名规范

- 包名、模块名、局部变量名、函数名
        全小写+下划线式驼峰
        example：this_is_var
- 全局变量
        全大写+下划线式驼峰
        example：GLOBAL_VAR
- 类名
        首字母大写式驼峰
        example：ClassName()
- 属性
        以单下划线开头，是弱内部使用标识，from M import * 时，将不会导入该对象
        以双下划线开头的变量名，主要用于类内部标识类私有，不能直接访问。模块中使用见上一条
        双下划线开头且双下划线截尾的命名方法尽量不要用，这是标识

In [12]:
# 字符编码
s='内置编码'
print(repr(s))
u=s.encode('utf8')
print(u)
print(type(u))
print(repr(u))

s='解码'


'内置函数'
b'\xe5\x86\x85\xe7\xbd\xae\xe5\x87\xbd\xe6\x95\xb0'
<class 'bytes'>
b'\xe5\x86\x85\xe7\xbd\xae\xe5\x87\xbd\xe6\x95\xb0'


## 数据结构

- list
    1. 列表当堆栈使用
        ```
        先进后出、后进先出
        .append()
        .pop()
        ```
    2. 列表当队列使用
        ```
        from collections import deque
        queue = deque([1, 2, 3, 4, 5])
        queue.append(6)
        
        -> deque([1, 2, 3, 4, 5, 6])
        ```
    3. 列表推导例
        名为 x 的变量在循环完毕后依然存在 @list-3
    4. del
        从列表中按给定的索引而不是值来删除一个子项
- tuple
- set
        集合是一个无序不重复元素的集
        union（联合），intersection（交），difference（差）和 sysmmetric difference（对称差集）
- dict
        ```
        d = {'abc': 123, 'defg': 456}
        c = dict(abc=123, defg=456)
        e = dict([('abc', 123), ('defg', 456)])

        ```
- 循环
    1. item()
    2. enumerate() 索引位置和对应值
    3. zip()       同时循环两个或更多的序列
    4. reversed()  逆向循环序列
    5. sorted()    不改动原序列生成一个新
    6. 若要在循环内部修改正在遍历的序列,建议您首先制作副本


In [2]:
# list-3.1
    a = []
    for x in range(10):
        a.append(x ** 2)
    print(x)
    print(a)

    b = list(map(lambda y: y ** 2, range(10)))
    print(b)
    c = [z ** 2 for z in range(10)]
    print(c)

9
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [3]:
# list-3.2
    matrix = [
        [1, 2, 3, 4],
        [5, 6, 7, 8],
        [9, 10, 11, 12],
    ]
    a = [[row[i] for row in matrix] for i in range(4)]
    print(a)

    b = []
    for i in range(4):
        b.append([row[i] for row in matrix])
    print(b)

    c = []
    for i in range(4):
        d = []
        for row in matrix:
            print('%s-->%s' % (i, row[i]))
            d.append(row[i])
        c.append(d)
    print(c)

[[1, 5, 9], [2, 6, 10], [3, 7, 11], [4, 8, 12]]
[[1, 5, 9], [2, 6, 10], [3, 7, 11], [4, 8, 12]]
0-->1
0-->5
0-->9
1-->2
1-->6
1-->10
2-->3
2-->7
2-->11
3-->4
3-->8
3-->12
[[1, 5, 9], [2, 6, 10], [3, 7, 11], [4, 8, 12]]


In [7]:
# set-1

a = set('abcdef')
b = set('abcxyz')
c = a - b
print(c)
d = a | b
print(d)
e = a & b
print(e)
f = a ^ b
print(f)

{'e', 'd', 'f'}
{'e', 'x', 'a', 'd', 'y', 'b', 'z', 'f', 'c'}
{'c', 'a', 'b'}
{'z', 'e', 'f', 'x', 'd', 'y'}


## 函数

- 作用域
    * L local
        局部作用域
    * E enclosing
        嵌套的父级函数的局部作用域，即包含此函数的上级函数的局部作用域
    * G globa
        全局变量，就是模块级别定义的变量
    * B built-in
        系统固定模块里面的变量，比如int, bytearray等
    * 搜索变量的优先级顺序依次
        作用域局部>外层作用域>当前模块中的全局>python内置作用域
        L--E--G--B      
- nonlocal
- global

- 继承
    1. isinstance(obj,int) 检查实例类型
            obj.__class__==int
    2. issubclass(bool,int) 检查类继承
    3. 搜索属性从父类继承的深度优先，左到右
    
- 装饰器
    1. 区分导入时执行和运行时执行
    2. 闭包和 nonlocal 声明
            method=method_name()
            method.__code__.co_freevars --> 延伸的值保存在内层函数的code属性中
        
            对数字、字符串、元组等不可变类型来说，只能读取，不能更新
            例如 count = count + 1，其实会隐式创建局部变量 count。 
            Python 3 引入了 nonlocal 声明，nonlocal 声明的变量赋予新值，闭包中保存的绑定会更新
    3. 二级嵌套
            当用装饰器调用函数时，返回的是一个未运行的函数,装饰器自身调用2次 @装饰器-2
    4. 多级嵌套
            @functools.wraps(func) 用法
    5. 单例运用
            字典里只放一个对象，后面不创建对象，及参数

- 内置函数

    1. __import__(name, globals=None, locals=None, fromlist=(), level=0) # 动态导入模块
        ```
        __db_type = 'Sqlite_Client'
        __import__(__db_type)
        
        --> <module 'Sqlite_Client' from '/home/xxxx/Project/python/pythonSpider/db/Sqlite_Client.py'>
            导入模块
        ```
    2. getattr(object, name, default=None) # 函数用于返回一个对象属性值
        ```
        __db_type = None
        if 'sqlite' == self.conf.db_type:
            __db_type = 'Sqlite_Client'
            self.client = getattr(__import__(__db_type), 'SqliteClient')()
            
        --> Sqlite_Client
            <module 'Sqlite_Client' from '/home/xxxx/Project/python/pythonSpider/db/Sqlite_Client.py'>
            <Sqlite_Client.SqliteClient object at 0x7f96d6972e10>
            
            创建SqliteClien()
        ```

In [14]:
x=int(2.9)
g_count=0 #G
def outer():
    e_count=1 #E
    def inner():
        l_count=2 #L
        print(e_count)
        pass
    inner() 
outer()
# local和enclosing是相对的，enclosing变量相对上层来说也是local
# def、class、lambda是可以引入新作用域的

1


In [6]:
# 装饰器-1
list_a = []
def deco(func):
    print('run deco is {}'.format(func))
    list_a.append(func)
    return func
@deco
def f1():
    print('f1')
@deco
def f2():
    print('f2')
def f3():
    print('f3')
if __name__ == '__main__':
    """
    在执行main之前，导入装饰器的函数即执行装饰器
    f3当其被调用时，才被创建
    """
    print('main')
    print('list', list_a)
    f1()
    f2()
    f3()
    print('list',list_a)

run deco is <function f1 at 0x7fcad0394e18>
run deco is <function f2 at 0x7fcad03946a8>
main
list [<function f1 at 0x7fcad0394e18>, <function f2 at 0x7fcad03946a8>]
f1
f2
f3
list [<function f1 at 0x7fcad0394e18>, <function f2 at 0x7fcad03946a8>]


In [9]:
# 装饰器-2

def local_count_error():
    count=2
    def nonlocal_count(value):
        count+=value
        return count
    return nonlocal_count

def local_count():
    count=2
    def nonlocal_count(value):
        nonlocal count
        count+=value
        return count
    return nonlocal_count
if __name__=='__main__':
    c=local_count()
    print(c(2))
    print(c(4))

4
8


In [7]:
# 装饰器-3

import time
import functools

def logger(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        f = func(*args, **kwargs)
        end = time.time()
        print('{} running time --> {}'.format(func.__name__, end - start))
        return f
    return wrapper

def logger(txt):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            start = time.time()
            f = func(*args, **kwargs)
            end = time.time()
            print('{}->tag {} running {}'.format(txt, func.__name__, end - start))
            return f

        return wrapper

    return decorator

@logger('three')
def run_time():
    for i in range(3):
        print('method is running {}'.format(i))
        time.sleep(1)
    print('it is ok')
    
if __name__=='__main__':
    run=run_time()
    
#     log=logger(run_time)
#     log()
    
    log_t=logger('log')(run_time)
    print(log_t.__name__)
    

method is running 0
method is running 1
method is running 2
it is ok
three->tag run_time running 3.002363443374634
run_time


In [ ]:
# 装饰器-5

def singleton_class(cls):
    """
    构建单例方法
    :param cls:
    :return:
    """
    instance = dict()
    @functools.wraps(cls)
    def get_instance(*args, **kwargs):
        if cls not in instance:
            instance[cls] = cls(*args, **kwargs)
        return instance[cls]

    return get_instance


@singleton_class
class WantSingle(object):
    def __init__(self, num):
        self.num = num

## Process



## Thread



## 异常

- raise
    1. raise Class
        Class 必须是 type 或其派生类的一个实例
    2. raise Class()

## IO

* 用户空间
* 内核空间
* 进程切换
* 进程阻塞

## Spider

- uillib.request
        自带的库
- beautifulsoup4        
- Selenium

- PhantomJS

        [PhantomJS官方地址](http://phantomjs.org/)
        [PhantomJS官方API](http://phantomjs.org/api/)
        [PhantomJS官方示例](http://phantomjs.org/examples/)
        [PhantomJS GitHub](https://github.com/ariya/phantomjs/)


## requests

## Jinja2

## pandas

## Numpy

In [ ]:
OpenStack架构、原理及其主要模块Nova，Cinder
了解常见的云平台备份技术及方案，如：freezer、smaug、snapshot等
一种主流的虚拟化平台，如：libvirt、qemu、kvm
